In [151]:
import pandas as pd
import numpy as np
from textFunctions import *
from datetime import datetime
import gc
from nltk import FreqDist
import lda
from ast import literal_eval
from gensim import corpora
import gensim
from gensim import models
from gensim.utils import tokenize
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import nltk
import pyLDAvis
import pyLDAvis.sklearn
import chainer
pyLDAvis.enable_notebook()
pd.set_option('display.max_rows', 999)

In [3]:
hs=pd.read_excel('headspace_0724.xlsx')

In [4]:
hs.date=hs.date.str.replace('\'', '')
hs.date=hs.date.str.replace('[', '')
hs.date=hs.date.str.replace(']', '')
hs.date=pd.to_datetime(hs['date'],format='%Y, %m, %d')
hs['texts']=hs['title']+hs['text']
hs.fillna('0',inplace=True)

In [5]:
hs.head()

,author,category,link,text,title,year,date,texts
0,Andy Puddicombe,HEALTH / MEDITATION,https://www.headspace.com/blog/2018/01/10/comp...,Editor’s Note: Below is an excerpt from Mindfu...,The complex relationship between your mind and...,2018,2018-01-10,The complex relationship between your mind and...
1,Mae Elizabeth Gurene,LIVING / MEDITATION,https://www.headspace.com/blog/2017/12/05/not-...,“I don’t care if I’m scared of a plastic bag. ...,“I realized this year that maybe I’m not as gr...,2017,2017-12-05,“I realized this year that maybe I’m not as gr...
2,Gemma Hartley,HEALTH / MEDITATION,https://www.headspace.com/blog/2017/11/27/boos...,I still remember the first week I tried medita...,"How to boost your brain power, no matter your age",2017,2017-11-27,"How to boost your brain power, no matter your ..."
3,Laura Riley,HEALTH / MEDITATION,https://www.headspace.com/blog/2017/11/06/mind...,A Mindful Pregnancy is a blog series that will...,A Mindful Pregnancy: Weeks 1-7,2017,2017-11-06,A Mindful Pregnancy: Weeks 1-7A Mindful Pregna...
4,George Bell,HEALTH / MEDITATION,https://www.headspace.com/blog/2017/11/04/medi...,The World Health Organization recently listed ...,What it’s like to meditate with depression,2017,2017-11-04,What it’s like to meditate with depressionThe ...


In [6]:
len(hs)

1013

In [7]:
hs['token'] = hs['text'].astype(str).apply(lambda x: tokenize(x))

In [9]:
def most_used(art, ngrams=1, mod=None, month='', site=''):
    '''Returns dict with counts, and the number of articles in a given dataframe.
    
    Parameters:
    - art: dataframe with articles
    - ngrams: n of n-grams (default: 1, words)
    - mod: dir to save files with articles (None if no saving necessary)
    - month: month for filename
    - site: site for filename
    '''
    word_dict = {} # save to it later
    print(art.shape[0], month, site) # number of articles
    if art.shape[0] == 0:
        return {}, 0 # we don't divide by it later; continue
    art['text'].fillna('', inplace=True) # get rid of nans, just in case. There should be none
    # and there are none in articles, but reddit doesn't work without this line
    time_start = datetime.now()
    art_sh = art[['text']].copy() # we don't need more columns
    del art # get rid of original df
    gc.collect() # clean memory, save RAM
    div = art_sh.shape[0]
    print(art_sh.shape[0]) # number of articles without nans
    
    # clean texts a bit using regexp
    # later we split by space (in function imported from textFunctions), so it's necessary
    art_sh['text'] = art_sh['text'].apply(lambda x: re.sub('\n', ' ', x))
    art_sh['text'] = art_sh['text'].apply(lambda x: re.sub('\t', ' ', x))

    # tokenize
    bigrams = False
    if ngrams == 2:
        bigrams = True
    art_sh['text'] = art_sh['text'].apply(lambda x: tokenize(x, bigrams=bigrams, words=True, lang='english'))

    # cleaning, quite slow, but necessary
    # this function is actually quite minimal to save time, if you need something else, you may want more preprocessing
    art_sh['text'] = art_sh['text'].apply(lambda x: cleaning(x, removePunctuation=True, removeNumbers=True, toLower=True, removeSpaces=True, stopwords=True, decodeDiacritics=False, shortwords=True)) 

    # frequency
    fdist = FreqDist([i for sublist in art_sh['text'].tolist() for i in sublist])
    x = dict(fdist)
    del fdist
    gc.collect()
    
    # we may need chunks later
    # word_dict = {k: x.get(k, 0) + word_dict.get(k, 0) for k in set(x) | set(word_dict)}
    
    # save tokenized articles
    if mod is not None:
        art_sh[['text']].to_csv(mod + '' + str(ngrams) + '.csv')

    print(datetime.now() - time_start)
    del art_sh
    gc.collect()
    return x, div

In [10]:
res = './res/'
for n in [1,2]:
    word_dfs_all = pd.DataFrame()
    # call the function defined above
    word_dict, div = most_used(hs, ngrams=n, mod=res)
            # convert result to df
    if div == 0:
        continue # avoid division by 0 and assigning column names to empty dataframes
    word_df = pd.DataFrame.from_dict(word_dict, orient='index')
    del word_dict
    gc.collect()
    # we need counts and frequencies
    word_df.columns = ['count']
    word_df['freq'] = word_df['count'] / div
    word_dfs_all = word_df.copy()
    print(word_dfs_all.shape)
    del word_df
    gc.collect()
    word_dfs_all.to_csv(res + 'freq_' + str(n) + '-all.csv')
    del word_dfs_all # we won't need it later
    gc.collect()

1013  
1013
0:00:13.847630
(26674, 2)
1013  
1013
0:00:18.570013
(270470, 2)


In [ ]:
# stop=pd.read_csv('stop-word-list.txt',header=0)
# stop=list(stop['stopwords'])
# len(stop)

In [12]:
freq=pd.read_csv('/Users/Michal_Palinski/Dropbox/Mindfulness/headspace/res/freq_1-all.csv')
freq.rename(columns={'Unnamed: 0': 'word'},inplace=True)

In [ ]:
# freqNS=freq[~freq.word.isin(stop)]

In [13]:
freq.sort_values('freq',ascending=False).to_excel('hsFreq2.xlsx')

In [14]:
freq.sort_values('freq',ascending=False)

,word,count,freq
22163,time,2745,2.709773
6549,like,2227,2.198421
25236,people,1878,1.853899
7213,just,1848,1.824284
14541,meditation,1830,1.806515
19672,feel,1736,1.713722
1870,life,1575,1.554788
7799,day,1521,1.501481
5787,says,1382,1.364265
6579,way,1337,1.319842


In [15]:
df=pd.read_excel('hsFreq2.xlsx')

In [16]:
df.head()

,word,count,freq
22163,time,2745,2.709773
6549,like,2227,2.198421
25236,people,1878,1.853899
7213,just,1848,1.824284
14541,meditation,1830,1.806515


In [17]:
freqBigr=pd.read_csv('/Users/Michal_Palinski/Dropbox/Mindfulness/headspace/res/freq_2-all.csv')
freqBigr.rename(columns={'Unnamed: 0': 'word'},inplace=True)

In [19]:
freqBigr.sort_values('freq',ascending=False)

,word,count,freq
157227,in the,2635,2.601185
75876,of the,2276,2.246792
145811,to be,1560,1.539980
115133,to the,1313,1.296150
24237,on the,1262,1.245805
184399,i was,1185,1.169793
94697,in a,998,0.985192
119418,can be,802,0.791708
28790,is a,800,0.789733
202021,with a,739,0.729516


In [20]:
important=['meditation',
 'feel',
 'mind',
 'mindfulness',
 'think',
 'know',
 'thoughts',
 'practice',
 'experience',
 'feeling',
 'focus',
 'feelings',
 'change',
 'present',
 'experiences',
 'study',
 'mindful',
 'attention',
 'research',
 'emotions',
 'insight',
 'life',
 'help',
 'anxiety',
 'brain',
 'body',
 'sleep',
 'stress',
 'health',
 'mental',
 'exercise',
 'results',
 'positive',
 'physical',
 'wellbeing',
 'people',
 'world',
 'social',
 'love',
 'relationship',
 'friends',
 'home',
 'family',
 'community',
 'myself',
 'personal',
 'self',
 'individual',
 'time',
 'day',
 'years',
 'moment',
 'long',
 'days',
 'minutes',
 'times',
'suffering']

In [23]:
s=hs['text']

In [83]:
insight=['comprehension','contemplation','realization']
health=['sickness','anxiety','wellbeing']
community=['relationship','colleague','community']
individual=['personal','myself','emotions']
productivity=['productivity','boost','results']

In [65]:
li=insight+health+community+individual

In [84]:
ins=s[s.str.contains('|'.join(insight))]
hea=s[s.str.contains('|'.join(health))]
com=s[s.str.contains('|'.join(community))]
ind=s[s.str.contains('|'.join(individual))]
pro=s[s.str.contains('|'.join(productivity))]

In [85]:
rins=ins.sample(5, random_state=1)
rhea=hea.sample(5, random_state=1)
rcom=com.sample(5, random_state=1)
rind=ind.sample(5, random_state=1)
rpro=pro.sample(5, random_state=1)

In [102]:
ins=hs[hs.index.isin(rins.index)]
hea=hs[hs.index.isin(rhea.index)]
com=hs[hs.index.isin(rcom.index)]
ind=hs[hs.index.isin(rind.index)]
pro=hs[hs.index.isin(rpro.index)]

In [103]:
ins['cat']='insight'
hea['cat']='health'
com['cat']='community'
ind['cat']='individual'
pro['cat']='productivity'

/Users/Michal_Palinski/miniconda3/envs/myPython/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/Michal_Palinski/miniconda3/envs/myPython/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/Michal_Palinski/miniconda3/envs/myPython/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

In [104]:
li=[ins,hea,com,ind,pro]
df=pd.concat(li)

In [106]:
df.to_excel('random_articles_keywords.xlsx')

## Topic modeling

In [107]:
corpus=hs['text'].tolist()
corpusStr=' '.join(corpus)

In [109]:
corpus=[x.lower() for x in corpus]

In [110]:
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = 'english',
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.5, 
                                min_df = 10)
dtm_tf = tf_vectorizer.fit_transform(corpus)
print(dtm_tf.shape)

(1013, 4077)


In [111]:
tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())
dtm_tfidf = tfidf_vectorizer.fit_transform(corpus)
print(dtm_tfidf.shape)

(1013, 4077)


In [143]:
# for TF DTM
lda_tf = LatentDirichletAllocation(n_components=9, random_state=0)
lda_tf.fit(dtm_tf)
# for TFIDF DTM
lda_tfidf = LatentDirichletAllocation(n_components=9, random_state=0)
lda_tfidf.fit(dtm_tfidf)

/Users/Michal_Palinski/miniconda3/envs/myPython/lib/python3.5/site-packages/sklearn/decomposition/online_lda.py:532: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
/Users/Michal_Palinski/miniconda3/envs/myPython/lib/python3.5/site-packages/sklearn/decomposition/online_lda.py:532: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=10,
             mean_change_tol=0.001, n_components=9, n_jobs=1,
             n_topics=None, perp_tol=0.1, random_state=0,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)

In [144]:
prep=pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer, mds='tsne')

In [145]:
ht=pyLDAvis.prepared_data_to_html(prep)

In [146]:
pyLDAvis.save_html(prep,'./docs/lda.html')

In [24]:
old_datetime = datetime.now()
ngrams = [(1,1)]
res = './res/'
month = 'test'

for ngram_art, ngram_compare in ngrams:
#     dataframe, sort by coef
#     take only these ngrams_compare which experience significant growth
    coefs_compare = pd.read_csv(res + 'freq_' + str(ngram_compare) + '-all.csv', index_col=0)
    coefs_compare = coefs_compare.loc[coefs_compare['freq'] < 0.5].sort_values('freq', ascending=False)
    
    coefs_art = pd.read_csv(res + 'freq_' + str(ngram_art) + '-all.csv', index_col=0)
    coefs_art = coefs_art.loc[coefs_art['freq'] < 0.5].sort_values('freq', ascending=False)

    # take only most common, growing ngrams
#     important_words = coefs_art.index[2:12].tolist()
    if ngram_art == 1:
        important_words = important
    elif ngram_art == 2:
        important_words = important_2
    
    cooc_all = []
    other_words = {}

    # 5000 most significant ngrams
    for i, row in coefs_compare[:2500].iterrows():
        other_words[i] = row['freq']
    
    cooc = {}
    print(month, datetime.now() - old_datetime)
    old_datetime = datetime.now()
        # use the fact that indices are the same
        # probably more clear would be joining by links
    
    dfs_art = pd.read_csv(res + str(ngram_art) + '.csv')
    dfs_art['text'] = dfs_art['text'].apply(lambda x: set(literal_eval(x)))
    dfs_art.rename(columns={'text': 'text_token_art'}, inplace=True)
    dfs_compare = pd.read_csv(res + str(ngram_compare) + '.csv')
    dfs_compare['text'] = dfs_compare['text'].apply(lambda x: set(literal_eval(x)))
    dfs_compare.rename(columns={'text': 'text_token_compare'}, inplace=True)
        
    dfs = pd.concat([dfs_art[['text_token_art']], dfs_compare[['text_token_compare']]], axis=1)
        
    article_word_count = {}
        
    # save RAM
    del dfs_art
    del dfs_compare
    gc.collect()
        
    for word in important_words:
        print(word)
        cooc[word] = {}
            # take only articles with the desired word
        df_site_cont = dfs.loc[dfs['text_token_art'].apply(lambda x: word in x)]
        article_word_count[word] = df_site_cont.shape[0]

        # we don't want iterating a lot of times over a dataframe
        # because it's slow, so first we assign zeroes to all words, and then add 1 if word exists
        for k in other_words.keys():
            cooc[word][k] = 0

        # iterate over articles, then after words
        for i, row in df_site_cont.iterrows():
            for k in other_words.keys():
                cooc[word][k] += k in row['text_token_compare']
                
        cooc_all.append(pd.DataFrame.from_dict(cooc))
    
    # concat
    cooc = pd.concat(cooc_all, axis=1)

    cooc = cooc.loc[:, ~cooc.columns.duplicated()]
    # three coefficients
    # frequency, count normalized by mean, count normalized by mean squared
    for i_word, word in enumerate(important_words):
        print('coef', word, 100*i_word/len(important_words), datetime.now() - old_datetime)
        old_datetime = datetime.now()
        if word not in cooc.columns:
            continue
        if article_word_count[word] > 0:
            cooc[word + '_freq'] = 100 * cooc[word] / article_word_count[word]
        else:
            cooc[word + '_freq'] = 0
                
    # count weighted values
    for i_word, word in enumerate(important_words):
        print('count', word, 100*i_word/len(important_words), datetime.now() - old_datetime)
        old_datetime = datetime.now()
        cooc[word + '_art_' + 'freq_weighted'] = 0
        cooc[word + '_art_' + 'freq_weighted_normalized'] = 0
        cooc[word + '_art_' + 'freq_weighted_normalized2'] = 0
        if word in cooc.columns: # avoid KeyErrors
            cooc[word + '_art_' + 'freq_weighted'] += cooc[word + '_freq']
            cooc[word + '_art_' + 'freq_weighted_normalized'] += cooc[word + '_freq']
            cooc[word + '_art_' + 'freq_weighted_normalized2'] += cooc[word +  '_freq']
        for word_compare, mean in other_words.items():
            cooc.loc[word_compare, word + '_art_' + 'freq_weighted_normalized'] = cooc.loc[word_compare, word + '_art_' + 'freq_weighted_normalized'] / mean
            cooc.loc[word_compare, word + '_art_' + 'freq_weighted_normalized2'] = cooc.loc[word_compare, word + '_art_' + 'freq_weighted_normalized2'] / mean**2

    cooc[[x for x in cooc.columns if 'weighted' in x]].to_csv(res + 'cooc' + str(ngram_art) + str(ngram_compare) + 'weighted2.csv')

test 0:00:00.603519
meditation
feel
mind
mindfulness
think
know
thoughts
practice
experience
feeling
focus
feelings
change
present
experiences
study
mindful
attention
research
emotions
insight
life
help
anxiety
brain
body
sleep
stress
health
mental
exercise
results
positive
physical
wellbeing
people
world
social
love
relationship
friends
home
family
community
myself
personal
self
individual
time
day
years
moment
long
days
minutes
times
suffering
coef meditation 0.0 0:12:33.956442
coef feel 1.7543859649122806 0:00:00.461468
coef mind 3.508771929824561 0:00:00.001296
coef mindfulness 5.2631578947368425 0:00:00.001298
coef think 7.017543859649122 0:00:00.001008
coef know 8.771929824561404 0:00:00.001861
coef thoughts 10.526315789473685 0:00:00.000952
coef practice 12.280701754385966 0:00:00.000961
coef experience 14.035087719298245 0:00:00.000968
coef feeling 15.789473684210526 0:00:00.001207
coef focus 17.54385964912281 0:00:00.001145
coef feelings 19.29824561403509 0:00:00.000906
coef c

In [25]:
cooc=pd.read_csv(res+'cooc11weighted2.csv')
cooc.rename(columns={'Unnamed: 0':'keywords'},inplace=True)

In [29]:
cooc.sort_values('suffering_art_freq_weighted_normalized',ascending=False)[['keywords','suffering_art_freq_weighted_normalized','suffering_art_freq_weighted']][:50]

,keywords,suffering_art_freq_weighted_normalized,suffering_art_freq_weighted
2180,suffering,978.350515,100.000000
1069,hundreds,492.987013,11.428571
436,concrete,406.714286,8.571429
1335,medications,406.714286,8.571429
125,ashamed,369.740260,8.571429
1471,observed,369.740260,8.571429
1792,reduces,325.371429,8.571429
2193,supported,322.789116,7.142857
1292,loving,316.333333,10.000000
1306,majority,316.333333,10.000000


In [ ]:
cooc.sort_values('relationship_art_freq_weighted_normalized',ascending=False)

In [ ]:
cooc.sort_values('stress_art_freq_weighted_normalized',ascending=False)

In [ ]:
import pandas as pd
import numpy as np
from functools import reduce
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.probability import FreqDist
from nltk import word_tokenize
from nltk.util import ngrams
from __future__ import print_function, unicode_literals
import spacy
from collections import defaultdict, Counter
import re
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from sklearn.externals import joblib
from __future__ import print_function
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.manifold import MDS
from scipy.cluster.hierarchy import ward, dendrogram
import mpld3
%matplotlib inline

In [ ]:
stopwords = nltk.corpus.stopwords.words('english')
stemmer = SnowballStemmer("english")

In [6]:
hs['texts']=hs['title']+hs['text']

In [7]:
hs.fillna('',inplace=True)

In [8]:
corpus=hs['texts'].tolist()
corpusStr=' '.join(corpus)

In [ ]:
# here I define a tokenizer and stemmer which returns the set of stems in the text that it is passed

def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [ ]:
totalvocab_stemmed = []
totalvocab_tokenized = []
for i in corpus:
    allwords_stemmed = tokenize_and_stem(i)
    totalvocab_stemmed.extend(allwords_stemmed)
    
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)

In [ ]:
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)

In [ ]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=200000,
                                 min_df=0.2, stop_words='english',
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)

print(tfidf_matrix.shape)

In [ ]:
terms = tfidf_vectorizer.get_feature_names()

In [ ]:
dist = 1 - cosine_similarity(tfidf_matrix)

In [ ]:
num_clusters = 3

km = KMeans(n_clusters=num_clusters)

%time km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

In [ ]:
joblib.dump(km,  'doc_cluster.pkl')
km = joblib.load('doc_cluster.pkl')
clusters = km.labels_.tolist()

In [ ]:
wp = { 'title': hs['title'].tolist(), 'text': hs['texts'].tolist(), 'cluster': clusters }
frame = pd.DataFrame(wp, index = [clusters] , columns = ['title', 'text', 'cluster'])

In [ ]:
print("Top terms per cluster:")
print()
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
for i in range(num_clusters):
    print("Cluster %d words:" % i, end='')
    for ind in order_centroids[i, :6]:
        print(' %s' % vocab_frame.ix[terms[ind].split(' ')].values.tolist()[0][0].encode('utf-8', 'ignore'), end=',')
    print()
    print()
    print("Cluster %d titles:" % i, end='')
    for title in frame.ix[i]['title'].values.tolist():
        print(' %s,' % title, end='')
    print()
    print()

In [ ]:
frame['cluster'].value_counts()

In [ ]:
MDS()

# two components as we're plotting points in a two-dimensional plane
# "precomputed" because we provide a distance matrix
# we will also specify `random_state` so the plot is reproducible.
mds = MDS(n_components=2, dissimilarity="precomputed", random_state=1)

pos = mds.fit_transform(dist)  # shape (n_components, n_samples)

xs, ys = pos[:, 0], pos[:, 1]

In [ ]:
#set up colors per clusters using a dict
cluster_colors = {0: '#1b9e77', 1: '#d95f02', 2: '#7570b3', 3: '#e7298a', 4: '#66a61e'}

#Cluster 0 words: b'says', b'feel', b'day', b'people', b'make', b'life',
#Cluster 1 words: b'mind', b'brain', b'feel', b'body', b'emotional', b'thoughts',
#Cluster 2 words: b'headspace', b'week', b'talk', b'live', b'world', b'people',
#Cluster 3 words: b'meditating', b'mind', b'headspace', b'day', b'practicing', b'feel',

#set up cluster names using a dict
cluster_names = {0: 'feel, people, life', 
                 1: 'mind, brain, body, emotional', 
                 2: 'talk, live, world', 
                 3: 'meditating, mind, practicing',
                 4: 'x'
                 }

In [ ]:
class TopToolbar(mpld3.plugins.PluginBase):
    """Plugin for moving toolbar to top of figure"""

    JAVASCRIPT = """
    mpld3.register_plugin("toptoolbar", TopToolbar);
    TopToolbar.prototype = Object.create(mpld3.Plugin.prototype);
    TopToolbar.prototype.constructor = TopToolbar;
    function TopToolbar(fig, props){
        mpld3.Plugin.call(this, fig, props);
    };

    TopToolbar.prototype.draw = function(){
      // the toolbar svg doesn't exist
      // yet, so first draw it
      this.fig.toolbar.draw();

      // then change the y position to be
      // at the top of the figure
      this.fig.toolbar.toolbar.attr("x", 150);
      this.fig.toolbar.toolbar.attr("y", 400);

      // then remove the draw function,
      // so that it is not called again
      this.fig.toolbar.draw = function() {}
    }
    """
    def __init__(self):
        self.dict_ = {"type": "toptoolbar"}

In [ ]:
#create data frame that has the result of the MDS plus the cluster numbers and titles
df = pd.DataFrame(dict(x=xs, y=ys, label=clusters, title=hs['title'])) 

#group by cluster
groups = df.groupby('label')

#define custom css to format the font and to remove the axis labeling
css = """
text.mpld3-text, div.mpld3-tooltip {
  font-family:Arial, Helvetica, sans-serif;
}

g.mpld3-xaxis, g.mpld3-yaxis {
display: none; }
"""

# Plot 
fig, ax = plt.subplots(figsize=(13,10)) #set plot size
ax.margins(0.03) # Optional, just adds 5% padding to the autoscaling

#iterate through groups to layer the plot
#note that I use the cluster_name and cluster_color dicts with the 'name' lookup to return the appropriate color/label
for name, group in groups:
    points = ax.plot(group.x, group.y, marker='o', linestyle='', ms=18, label=cluster_names[name], mec='none', color=cluster_colors[name])
    ax.set_aspect('auto')
    labels = [i for i in group.title]
    
    #set tooltip using points, labels and the already defined 'css'
    tooltip = mpld3.plugins.PointHTMLTooltip(points[0], labels,
                                       voffset=10, hoffset=10, css=css)
    #connect tooltip to fig
    mpld3.plugins.connect(fig, tooltip, TopToolbar())    
    
    #set tick marks as blank
    ax.axes.get_xaxis().set_ticks([])
    ax.axes.get_yaxis().set_ticks([])
    
    #set axis as blank
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)

    
ax.legend(numpoints=1) #show legend with only one dot
# ax.legend(loc="upper left")

mpld3.display() #show the plot

#uncomment the below to export to html
# html = mpld3.fig_to_html(fig)
# print(html)

In [ ]:
### doc2vec

In [35]:
import gensim
LabeledSentence = gensim.models.doc2vec.LabeledSentence

In [16]:
hs2=hs['texts']

In [19]:
hs2=pd.DataFrame({'texts':hs2.values})

In [23]:
i=0
for index, row in hs2.iterrows():
    if i > len(hs2):
        break
    else:
        f = open('./textdir/'+'hs'+str(i)+'.txt', 'w')
        f.write(row[0])
        f.close()
        i+=1

In [42]:
hs.to_csv('hs.csv')

In [30]:
110/200

0.55

In [31]:
0.5/0.55

0.9090909090909091

In [32]:
0.5/110/200

2.2727272727272726e-05

In [34]:
110/200

0.55

In [35]:
0.5/0.55

0.9090909090909091